In [1]:
updated_expressway_mapping = {
    "1001": ("KPE", "View from KPE/ECP"),
    "1002": ("KPE", "View from Kallang Bahru"),
    "1003": ("KPE", "View from KPE/PIE"),
    "1004": ("KPE", "View from Kallang Way Flyover"),
    "1005": ("KPE", "View from Defu Flyover"),
    "1006": ("KPE", "View from Tampines Flyover"),
    "1501": ("MCE", "View from Maxwell Road"),
    "1502": ("MCE", "View from Marina Boulevard/Marina Coastal Drive"),
    "1503": ("MCE", "View from MCE 1.02km"),
    "1504": ("MCE", "View from MCE/ECP"),
    "1505": ("MCE", "View from Marina Boulevard"),
    "1111": (
        "Loyang Ave / Tanah Merah Coast Road",
        "View from TPE(PIE), Exit 2 to Loyang Ave",
    ),
    "1112": (
        "Loyang Ave / Tanah Merah Coast Road",
        "View from TPE(PIE), Tampines Viaduct",
    ),
    "1113": (
        "Loyang Ave / Tanah Merah Coast Road",
        "View from Tanah Merah Coast Road towards Changi",
    ),
    "1701": ("CTE", "View from Moulmein Flyover"),
    "1702": ("CTE", "View from Braddell Flyover"),
    "1703": ("CTE", "View from St George Road"),
    "1704": ("CTE", "View from Entrance from Chin Swee Road"),
    "1705": ("CTE", "View from Ang Mo Kio Ave 5 Flyover"),
    "1706": ("CTE", "View from Yio Chu Kang Flyover"),
    "1707": ("CTE", "View from Bukit Merah Flyover"),
    "1709": ("CTE", "View from Exit 6 to Bukit Timah Road"),
    "1711": ("CTE", "View from Ang Mo Kio Ave 1 Flyover"),
    "2701": ("Woodlands Checkpoint", "View from Woodlands Causeway (Towards Johor)"),
    "2702": ("Woodlands Checkpoint", "View from Woodlands Checkpoint (Towards BKE)"),
    "2703": ("BKE", "View from Chantek Flyover"),
    "2704": ("BKE", "View from Woodlands Flyover (Towards Checkpoint)"),
    "2705": ("BKE", "View from Dairy Farm Flyover"),
    "2706": ("BKE", "View from after KJE Exit"),
    "2707": ("BKE", "View from Mandai Rd Entrance"),
    "2708": ("BKE", "View from Exit 5 to KJE (Towards Checkpoint)"),
    "3702": ("ECP", "View from Entrance from PIE"),
    "3704": ("ECP", "View from Entrance from MCE"),
    "3705": ("ECP", "View from Exit 2A to Changi Coast Road"),
    "3793": ("ECP", "View from Laguna Flyover"),
    "3795": ("ECP", "View from Marine Parade Flyover"),
    "3796": ("ECP", "View from Tanjong Katong Flyover"),
    "3797": ("ECP", "View from Tanjong Rhu"),
    "3798": ("ECP", "View from Benjamin Sheares Bridge"),
    "4703": ("Tuas Checkpoint", "View from Second Link at Tuas"),
    "4713": ("Tuas Checkpoint", "View from Tuas Checkpoint"),
    "4701": ("AYE", "View from Towards Alexandra Road"),
    "4702": ("AYE", "View from Keppel Viaduct"),
    "4704": ("AYE", "View from Lower Delta Road"),
    "4705": ("AYE", "View from Entrance from Yuan Ching Rd"),
    "4706": ("AYE", "View from Near NUS"),
    "4707": ("AYE", "View from Entrance from Jln Ahmad Ibrahim"),
    "4708": ("AYE", "View from Near Dover Drive"),
    "4710": ("AYE", "View from Towards Pandan Gardens"),
    "4712": ("AYE", "View from After Tuas West Road"),
    "4714": ("AYE", "View from Near West Coast Walk"),
    "4716": ("AYE", "View from Entrance from Benoi Rd"),
    "4798": ("Sentosa Gateway", "View from Sentosa Gateway (Towards Telok Blangah)"),
    "4799": ("Sentosa Gateway", "View from Sentosa Gateway (Towards Sentosa)"),
    "5794": ("PIE", "View from Bedok North"),
    "5795": ("PIE", "View from Eunos Flyover"),
    "5797": ("PIE", "View from Paya Lebar Flyover"),
    "5798": ("PIE", "View from Kallang Way"),
    "5799": ("PIE", "View from Woodsville Flyover"),
    "6701": ("PIE", "View from Kim Keat Link"),
    "6703": ("PIE", "View from Thomson Flyover"),
    "6704": ("PIE", "View from Mount Pleasant"),
    "6705": ("PIE", "View from Adam Road"),
    "6706": ("PIE", "View from Bukit Timah Expressway"),
    "6708": ("PIE", "View from Nanyang Flyover"),
    "6710": ("PIE", "View from Entrance from Jalan Anak Bukit"),
    "6711": ("PIE", "View from Entrance to PIE from ECP Changi"),
    "6712": ("PIE", "View from Exit 27 to Clementi Ave 6"),
    "6713": ("PIE", "View from Entrance from Simei Ave"),
    "6714": ("PIE", "View from Exit 35 to KJE"),
    "6715": ("PIE", "View from Hong Kah Flyover"),
    "6716": ("PIE", "View from Tuas Flyover"),
    "7791": ("TPE", "View from Upp Changi Flyover Towards PIE"),
    "7793": ("TPE", "View from Tampines Ave 10 Entrance"),
    "7794": ("TPE", "View from TPE(KPE) Exit"),
    "7795": ("TPE", "View from Entrance To Tampines Flyover"),
    "7796": ("TPE", "View from Exit to Punggol Flyover"),
    "7797": ("TPE", "View from Seletar West Link"),
    "7798": ("TPE", "View from Seletar Flyover"),
    "8701": ("KJE", "View from Choa Chu Kang West Flyover"),
    "8702": ("KJE", "View from Exit To BKE"),
    "8704": ("KJE", "View from Entrance From Choa Chu Kang Dr"),
    "8706": ("KJE", "View from Tengah Flyover"),
    "9701": ("SLE", "View from Lentor Flyover"),
    "9702": ("SLE", "View from Upp Thomson Flyover"),
    "9703": ("SLE", "View from SLE(BKE) Exit"),
    "9704": ("SLE", "View from Ulu Sembawang Flyover"),
    "9705": ("SLE", "View from Marsiling Flyover"),
    "9706": ("SLE", "View from Mandai Flyover"),
}

In [2]:
import requests
# import pandas as pd
from transformers import pipeline
from PIL import Image, ImageDraw
from io import BytesIO
from database import SessionLocal, TrafficData
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import base64

checkpoint = "google/owlv2-base-patch16-ensemble"
detector = pipeline(model=checkpoint, task="zero-shot-object-detection")

def fetch_traffic_images():
    url = "https://api.data.gov.sg/v1/transport/traffic-images"
    response = requests.get(url)
    return response.json()
    
def encode_image_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')
    
def draw_boxes_on_image(image, predictions):
    draw = ImageDraw.Draw(image)
    for p in predictions:
        box = p['box']
        label = p['label']
        score = p['score']

        xmin, ymin, xmax, ymax = box.values()
        draw.rectangle((xmin, ymin, xmax, ymax), outline='red', width=1)
        draw.text((xmin, ymin), f'{label}: {round(score, 2)}', fill='white')

    return image

def congestion_meter(number_of_cars):
    if number_of_cars >= 20:
        return 'congested'
    if number_of_cars >= 15:
        return 'slightly congested'
    if number_of_cars >= 10:
        return 'normal'
    if number_of_cars >= 5:
        return 'better than normal'
    else:
        return 'basically no cars'

def detect_and_transform(camera):
    camera_id = camera['camera_id']
    timestamp = datetime.fromisoformat(camera['timestamp'])
    expressway, camera_description = updated_expressway_mapping.get(camera_id, ('Unknown', 'Unknown')) ## to be changed later
    image_url = camera['image']

    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content)).convert('RGB')

    predictions = detector(image=image, candidate_labels=['car'])
    annotated_image = draw_boxes_on_image(image, predictions)

    annotated_image_base64 = encode_image_base64(annotated_image)
    print('saved')

    number_of_cars = sum(1 for item in predictions if item['label'] == 'car' and item['score'] > 0.12)
    description = congestion_meter(number_of_cars)

    return {
        "camera_id": camera_id,
        "camera_description": camera_description,
        "timestamp": timestamp,
        "expressway": expressway,
        "image_url": image_url,
        "annotated_image_url": annotated_image_base64,
        "number_of_cars": number_of_cars,
        "description": description
    }

def transform_and_store(expressway_name):
    traffic_data = fetch_traffic_images()
    cameras = traffic_data['items'][0]['cameras']

    filtered_cameras = [camera for camera in cameras if updated_expressway_mapping.get(camera['camera_id'], (None,))[0] == expressway_name]

    with ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(detect_and_transform, filtered_cameras))

    # df = pd.DataFrame(results)
    # print(df)

    db = SessionLocal()
    try:
        for row in results:
            traffic_data = TrafficData(**row)
            db.add(traffic_data)
        db.commit()
    finally:
        db.close()

    return results

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
result = transform_and_store('BKE')

saved
saved
saved
saved
saved
saved
